In [27]:
import pathlib

import requests
import networkx as nx
import geopandas as gpd
import pandas as pd
import shapely.geometry
import shapely.wkt

import dtv_backend.network.build_network_functions as bnf


version = '0.3'
output_dir = pathlib.Path('result')


# Add bathymetry information
TODO: split bathymetry in separate file/notebook
In this section we will make a few adjustments to the network. 
* We will rename the classes to short names for shapefile compatibility
* We will upload the shapefile to GEE and add extra information (bathymetry, maybe later other external variables)
* We will download the new enriched data and continue from there.

In [3]:
# read the networks
network = nx.read_gpickle(output_dir / f'network_digital_twin_v{version}.pickle')

In [4]:

shp_network = bnf.strip_geometries(bnf.rename_keys(network))


In [31]:
edges_df = nx.to_pandas_edgelist(shp_network)

geometry = edges_df['Wkt'].apply(shapely.wkt.loads)
edges_gdf = gpd.GeoDataFrame(edges_df, geometry=geometry, crs=4326)

nodes_df = pd.DataFrame(shp_network.nodes.values(), index=shp_network.nodes.keys())
geometry = nodes_df['Wkt'].apply(shapely.wkt.loads)
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry=geometry, crs=4326)



In [34]:

result_dir =  output_dir / f'network_digital_twin_v{version}-intermediate'
result_dir.mkdir(exist_ok=True)

edges_gdf.to_file(result_dir / 'edges.shp')
nodes_gdf.to_file(result_dir / 'nodes.shp')

!open $result_dir

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [38]:
# Copy the resulting shapefiles to the google cloud
!gsutil cp -r {result_dir} gs://slr/bodemhoogte/{result_dir.name}

Copying file://result/network_digital_twin_v0.3-intermediate/edges.dbf [Content-Type=application/octet-stream]...
Copying file://result/network_digital_twin_v0.3-intermediate/nodes.prj [Content-Type=application/octet-stream]...
Copying file://result/network_digital_twin_v0.3-intermediate/edges.shp [Content-Type=application/octet-stream]...
Copying file://result/network_digital_twin_v0.3-intermediate/edges.cpg [Content-Type=application/octet-stream]...
/ [4 files][ 26.7 MiB/ 26.7 MiB]  227.3 KiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://result/network_digital_twin_v0.3-intermediate/edges.shx [Content-Type=application/octet-stream]...
Copying file://result/network_digital_twin_v0.3-intermediate/nodes.shx [Content-Type=application/

In [39]:
!gsutil ls gs://slr/bodemhoogte/{result_dir.name}/edges.shp

gs://slr/bodemhoogte/network_digital_twin_v0.3-intermediate/edges.shp


In [43]:
!earthengine upload table --wait --asset_id=projects/deltares-rws/dmc/edges-{version.replace('.', '_')} gs://slr/bodemhoogte/{result_dir.name}/edges.shp


/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "
*** Earth Engine *** FINAL DEADLINE: ee.Authenticate will fail after 2022-06-06. Please upgrade. https://developers.google.com/earth-engine/guides/python_install
Started upload task with ID: MYTHEUM76DG64NH2324RYV4K
Waiting for the upload task to complete...
[16:35:58] Current state for task MYTHEUM76DG64NH2324RYV4K: RUNNING
[16:36:29] Current state for task MYTHEUM76DG64NH2324RYV4K: RUNNING
Task MYTHEUM76DG64NH2324RYV4K ended at state: COMPLETED after 72.22 seconds


# Add bathymetry info
Now that the edges are uploaded to an asset we can add the  [bathymetry](https://maps.rijkswaterstaat.nl/geoweb55/index.html?viewer=Bathymetrie_Nederland) information. Based on thescript [fis-bathy](https://code.earthengine.google.com/d9250d62af8146d05cab278aa83df877) we can now intersect the bathymetry with the gometry. This script will result in a new file, which we can then download from a bucket. 


In [82]:
import ee
ee.Initialize()

bathy = ee.ImageCollection("projects/deltares-rws/dmc/bodemhoogte_1mtr")
edges = ee.FeatureCollection(f"projects/deltares-rws/dmc/edges-{version.replace('.', '_')}")

RESOLUTION = 1

bathy_lat = bathy.filterMetadata('system:index', 'ends_with', 'LAT')
bathy_nap = bathy.filterMetadata('system:index', 'ends_with', 'NAP')

nap = bathy_nap.mosaic()
lat = bathy_lat.mosaic()
bands = nap.rename('nap').addBands(lat.rename('lat'))
reducer = ee.Reducer.percentile([0, 5, 10, 50, 90, 95, 100])
reducer = reducer.combine(**{
  "reducer2": ee.Reducer.mean(),
  "sharedInputs": True
})
reducer = reducer.combine(**{
  "reducer2": ee.Reducer.stdDev(),
  "sharedInputs": True,
})
reducer = reducer.combine(**{
  "reducer2": ee.Reducer.count(),
  "sharedInputs": True
})

result = bands.reduceRegions(edges, reducer, RESOLUTION)

task = ee.batch.Export.table.toCloudStorage(**{
  "collection": result, 
  "description": 'fis-bodemhoogte', 
  "bucket": 'slr', 
  "fileNamePrefix": f'bodemhoogte/edges_{version}_with_bathy', 
  "fileFormat": 'GeoJSON'
})
task.start()
task.id

'U77ORHZGD427A45BLBK5YFNK'

In [95]:
# wait for task to finish (this can take an hour or longer...)
# !earthengine task wait {task.id}
!earthengine task info QXQAQF7BG4P47MHTTJSM2MDG

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/jax/_src/lib/__init__.py:34: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "
QXQAQF7BG4P47MHTTJSM2MDG:
  State: RUNNING
  Type: Export.table
  Description: fis-bodemhoogte
  Created: 2022-06-19 16:45:46.651000
  Started: 2022-06-19 16:45:51.215000
  Updated: 2022-06-19 17:15:51.233000


In [92]:
result_path = f'gs://slr/bodemhoogte/edges_{version}_with_bathy'
!gsutil ls {result_path}

CommandException: One or more URLs matched no objects.
